<a href="https://colab.research.google.com/github/Processuales/FastOB_API/blob/main/FastOoba_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎵 Run Silent Audio Player { display-mode: "form" }

#@markdown Press play on the audio player that appears below. This will keep the Colab tab alive and prevent Google from disconnecting you for inactivity.
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title Connect to Google Drive || https://drive.google.com/drive/u/0/my-drive { display-mode: "form" }
#@markdown <| Make sure to have atleast 8GB of free space |>
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

shared_drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

def message(msg, style, wdth):
  message = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
  display(message)

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if shared_drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  main_path="/content/gdrive/Shareddrives/"+shared_drive
else:
  main_path="/content/gdrive/MyDrive/"

clear_output()
message('\u2714 Connected','success', '50px')

if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using temporary colab storage ...')
  main_path="/content/"
  shared_drive=""

if shared_drive != "" and not os.path.exists("/content/gdrive/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  main_path="/content/gdrive/MyDrive/"

#@markdown ---

folder_name = "Text-Gen" #@param {type :"string"}
text_gen_dir = f"{main_path}{folder_name}"
#@markdown - Crates a folder by this name if it does not exist already.

if not os.path.exists(text_gen_dir):
    os.makedirs(text_gen_dir)
    print(f"[0;32mCreated Folder at : {text_gen_dir}/")




In [ ]:
#@title Initialize Oobabooga Text-Web-UI || https://github.com/oobabooga/text-generation-webui
from IPython.display import clear_output
Update_Every_Execution = True #@param {type:"boolean"}
Download_Branch = False #@param {type:"boolean"}
custom_web_path = "" #@param {type:"string"}
#@markdown - You can leave this empty
#@markdown ---
use_cloudflare_public_api = True #@param {type :"boolean"}



%cd {text_gen_dir}

def message(msg, style, wdth):
  message = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
  display(message)

def main_web():
  if not os.path.exists(os.path.join(text_gen_dir, "text-generation-webui")): #
    !git clone https://github.com/oobabooga/text-generation-webui
    %cd {text_gen_dir}/text-generation-webui
  else:
    %cd {text_gen_dir}/text-generation-webui
    if Update_Every_Execution:
      !git pull origin

def branch_web():
  if not os.path.exists(os.path.join(text_gen_dir, "text-generation-webui")): #
    !git clone -b v2.5 https://github.com/camenduru/text-generation-webui
    %cd {text_gen_dir}/text-generation-webui
  else:
    %cd {text_gen_dir}/text-generation-webui
    if Update_Every_Execution:
      !git pull

def custom_web(path_to_web):
  if not os.path.exists(os.path.join(text_gen_dir, "text-generation-webui")): #
    !git clone {path_to_web}
    %cd {text_gen_dir}/text-generation-webui
  else:
    %cd {text_gen_dir}/text-generation-webui
    if Update_Every_Execution:
      !git pull

if custom_web_path == "":
  if not Download_Branch:
    main_web()
  else:
    branch_web()
else:
  custom_web(custom_web_path)

!pip install -q -r requirements.txt
!pip install xformers
if use_cloudflare_public_api:
  !pip install flask-cloudflared

!echo "dark_theme: true" > {text_gen_dir}/settings.yaml
!echo "chat_style: wpp" >> {text_gen_dir}/settings.yaml
!echo "mode: 'instruct'" >> {text_gen_dir}/settings.yaml


script_file_path = os.path.join(text_gen_dir, "text-generation-webui/extensions/api/script.py")

serveo_api_script = """
import extensions.api.blocking_api as blocking_api
import extensions.api.streaming_api as streaming_api
from modules import shared
import asyncio
import multiprocessing
import os

def runserv():
   os.system('ssh -o ServerAliveInterval=60 -R 80:127.0.0.1:5000 serveo.net')

def setup():
   blocking_api.start_server(shared.args.api_blocking_port, share=shared.args.public_api)
   streaming_api.start_server(shared.args.api_streaming_port, share=shared.args.public_api)

   proc = multiprocessing.Process(target=runserv)
   proc.start()
"""

cloudflare_api_script = """
import time

import extensions.api.blocking_api as blocking_api
import extensions.api.streaming_api as streaming_api
from modules import shared

def setup():
    blocking_api.start_server(shared.args.api_blocking_port, share=shared.args.public_api, tunnel_id=shared.args.public_api_id)
    time.sleep(5)
    streaming_api.start_server(shared.args.api_streaming_port, share=shared.args.public_api, tunnel_id=shared.args.public_api_id)
"""

with open(script_file_path, "w") as script_file:
    if use_cloudflare_public_api:
      script_file.write(cloudflare_api_script)
    else:
      script_file.write(serveo_api_script)

%cd {text_gen_dir}/text-generation-webui
print(f"[0;32mUpdated File : {script_file_path}")

if not use_cloudflare_public_api:
  !mkdir -p ~/.ssh && echo "Host *" > ~/.ssh/config && echo " StrictHostKeyChecking no" >> ~/.ssh/config
clear_output()
print(f"[0;32mUpdated File : {script_file_path}")
message('\u2714 Downloaded','success', '50px')

In [ ]:
#@title Download Models || https://huggingface.co/TheBloke
from IPython.display import clear_output

model_path = "TheBloke/Wizard-Vicuna-13B-Uncensored-SuperHOT-8K-GPTQ" #@param {type:"string"}
branch_path = "main" #@param {type:"string"}
model_name = model_path.split('/')[-1]
#@markdown ---
#@markdown --> Ignore if downloading a GPTQ or AWQ model from TheBloke <--
custom_install_path = "" #@param {type:"string"}
custom_file_name = custom_install_path.split('/')[-1]
#@markdown - Run cell multiple times if you have prerequisites for the model.
#@markdown - Right click on model download link and copy link.
#@markdown - Model Path is still required
#@markdown ---
#@markdown ---
#@markdown Recommended Models (and their non-superHOT alternative) :
#@markdown - TheBloke/Wizard-Vicuna-13B-Uncensored-SuperHOT-8K-GPTQ
#@markdown - TheBloke/Nous-Hermes-13B-SuperHOT-8K-GPTQ
#@markdown - TheBloke/Chronos-Hermes-13B-SuperHOT-8K-GPTQ
#@markdown - TheBloke/Pygmalion-13B-SuperHOT-8K-GPTQ
#@markdown - TheBloke/airoboros-l2-13b-gpt4-2.0-GGML [Not Tested]


%cd {text_gen_dir}
!apt-get -y install -qq aria2
%cd {text_gen_dir}/text-generation-webui

def message(msg, style, wdth):
  message = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
  display(message)

def download():
  if model_path == model_name or model_path.count("/") > 1:
    clear_output()
    message('\u2718 Incorrect Model Path','danger', "250px")
    return

  if branch_path == "" and custom_install_path == "":
    clear_output()
    message('\u2718 Please enter a branch (ex. main)','danger', "250px")
    return

  files = ["config.json","generation_config.json","special_tokens_map.json","tokenizer.model","tokenizer_config.json","model.safetensors"]

  for FILE in files:
    if FILE == "tokenizer.model" or FILE == "model.safetensors":
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model_path}/resolve/{branch_path}/{FILE} -d {text_gen_dir}/text-generation-webui/models/{model_name} -o {FILE}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model_path}/raw/{branch_path}/{FILE} -d {text_gen_dir}/text-generation-webui/models/{model_name} -o {FILE}

  clear_output()
  message('\u2714 Downloaded','success', '50px')


def custom_download():
  if model_path == model_name or model_path.count("/") > 1:
    message('\u2718 Wrong Path','danger', "250px")
    return

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {custom_install_path} -d {text_gen_dir}/text-generation-webui/models/{model_name} -o {custom_file_name}
  clear_output()
  message('\u2714 Downloaded','success', '50px')

if custom_install_path == "":
  download()
else:
  if custom_file_name != "":
    custom_download()
  else:
    message('\u2718 No File Name','danger', "250px")



In [ ]:
#@title Launch text-generation-webui || Reommended with SillyTavern
import sys
from IPython.display import clear_output

model_load =  "Wizard-Vicuna-13B-Uncensored-SuperHOT-8K-GPTQ" #@param {type:"string"}
public_link = False #@param {type:"boolean"}
public_api = True #@param {type:"boolean"}
max_seq_length = 8192 #@param {type:"slider", min:0, max:8192, step:512}
compress_pos_embed = 4 #@param {type:"slider", min:0, max:4, step:1}
#@markdown ---
model_backend = "ExLlama_hf" #@param ["ExLlama", "ExLlama_hf", "AutoGPTQ", "Llamacpp", "Llamacpp_hf", "rwkv","ctransformers"]
gpu_layers = 43 #@param {type:"slider", min:0, max:43, step:1}
offload_max_layers = False #@param {type:"boolean"}

#@markdown ---
groupsize_128 = True #@param {type:"boolean"}
load_4bit_models = True#@param {type:"boolean"}
load_in_8bit = False #@param {type:"boolean"}
use_xformers = False #@param {type:"boolean"}
max_gpu_mem = 0 #@param {type:"slider", min:0, max:14000, step:500}
#@markdown - Set to 0 to disable this

%cd {text_gen_dir}/text-generation-webui


if offload_max_layers:
  gpu_layers = 1000000000

if not public_link and not public_api:
  print("[0;31mModel Will Not Load")
  sys.exit(1)


params = set()

if public_link:
  params.add('--share')

if public_api:
  params.add('--public-api')

params.add(f'--settings {text_gen_dir}/settings.yaml')
params.add(f'--model {text_gen_dir}/text-generation-webui/models/{model_load}')
params.add(f'--loader {model_backend}')
params.add(f'--max_seq_len {max_seq_length}')
params.add(f'--compress_pos_emb {compress_pos_embed}')


if model_backend not in ["ExLlama", "ExLlama_hf", "AutoGPTQ"]:
  params.add(f'--n-gpu-layers {gpu_layers}')

if max_gpu_mem != 0:
  params.add(f'----gpu-memory {max_gpu_mem}Mib')

if load_4bit_models:
  params.add('--wbits 4')

if groupsize_128:
  params.add('--groupsize 128')

if use_xformers:
  params.add('--xformers')

clear_output()
!python server.py {' '.join(params)}


